In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from numpy import array
import cv2

#query = gesuchtes Objekt
#ref = Hintergrund
#Calculate Color Distance form every pixel in the actual Background sector

#TODO: 
#-shrink images b4 calc to reduce time
#-better procedure for calculating the distance not only adding them together
#-dominant color?


#-----------------------------------------------------------------------------------------------------
def getMeanRGB(partRefArray):
    width, height = query.size
    saveDistR = []
    saveDistG = []
    saveDistB = []
    rdist = 0
    gdist = 0
    bdist = 0
    
    for x in range(width):
        for y in range(height):
            #if alpha = 0 dont calculate that pixel -> transparent (because query image must exempted)
            if queryArray[x][y][3] > 0:
                #calculate dist for r g and b
                    toSaveR = abs(partRefArray[y][x][0] - queryArray[y][x][0])
                    toSaveG = abs(partRefArray[y][x][1] - queryArray[y][x][1])
                    toSaveB = abs(partRefArray[y][x][2] - queryArray[y][x][2])

                    saveDistR.append(toSaveR)
                    saveDistG.append(toSaveG)
                    saveDistB.append(toSaveB)
                    
    for x in range(len(saveDistR)):
        rdist = rdist + saveDistR[x]
        gdist = gdist + saveDistG[x]
        bdist = bdist + saveDistB[x]

    #mean dist for every color channel, maybe add weights connected with user input slider for every channel??
    rdistfinal = rdist/len(saveDistR)
    gdistfinal = gdist/len(saveDistG)
    bdistfinal = bdist/len(saveDistB)
    
    return [rdistfinal,gdistfinal,bdistfinal]
#----------------------------------------------------------------------------------------------------------------------

#load images
query = Image.open("flower100.png")
ref = Image.open("fire_01.jpg")

#write image to array
queryArray = array(query, dtype='int64')
refArray = array(ref, dtype='int64')

#calculate mean of whole ref
refmean = np.mean(ref)

#image width and height / 100  - 3000x4000/querysize, if image is 1620x1830 and x=16 y=18 the rest will not be calculated
stepsx = 30
stepsy = 20
#query image size 100x100
querysize = 100


width, height = ref.size
result = np.zeros([stepsx*stepsy+1 ,4])
counter = 0

print("Calculating Color Distance")

for x in range(stepsx):
    for y in range(stepsy):
        #slice 100x100 array out of whole ref image
        partRefArray = refArray[int(y*querysize):int(y*querysize+querysize),int(x*querysize):int(x*querysize+querysize)]
        
        #experimental -> maybe input dominant color calculation from david here
        partRefArrayMean = np.mean(partRefArray)
        meandist = abs(partRefArrayMean-refmean)
        if meandist > 40:
            counter = counter + 1
            #print("skip, meandist is too high: ",meandist)
            result[counter][0] = np.nan
            result[counter][1] = np.nan
            result[counter][2] = np.nan
            
            
        else:
        
            getMeanRGB(partRefArray)
        
            r, g, b = getMeanRGB(partRefArray)
            counter = counter +1
            result[counter][0] = r
            result[counter][1] = g
            result[counter][2] = b
            
print("Iterated over ", counter,"steps")
print("Done!")

resultlist = []

#add r+g+b/size ....for now no time to optimize
for i in range(counter):
    resultlist.append(result[i+1][0] + result[i+1][1] + result[i+1][2])
    
#get lowest distance
lowest = min(resultlist)
index = resultlist.index(lowest)

#print(resultlist)
print("lowest result:",lowest)
print("at index of list",index,"/",counter)

#calculate offset coords for query based on array index
offsetx = int(index/stepsy)*querysize
print("offset width: ",offsetx)

offsety= int(index % stepsy)*querysize
print("offset height: ",offsety)

#add both picture together and save them
r, g, b, a = query.split()
mask = Image.merge("L", (a,))
ref.paste(query, (offsetx, offsety), mask)
ref.save("result.png", "PNG")


Calculating Color Distance
Iterated over  600 steps
Done!
lowest result: 224.332369299
at index of list 490 / 600
offset width:  2400
offset height:  1000
